In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# 1. Análisis Sociodemográfico de Pacientes de Consultas

En este análisis se quiere analizar el perfil sociodemográfico de los pacientes que vienen a
realizar una consulta ambulatoria al INT. Se quieren resolver las siguientes preguntas:

- ¿Cuál es el rango etario de los pacientes atendidos?
- ¿Cuál es el sexo de los pacientes atendidos?
- ¿Cuál es la previsión de los pacientes atendidos?
- ¿Cuál es la región/comuna de los pacientes atendidos?
- ¿Cuál es el servicio de salud de los pacientes atendidos?

Además, se quiere responder cada unad e estas preguntas para cada una de las especialidades de
las consultas, diagnósticos y por tipo de consulta.


In [3]:
df_track = pd.read_csv("../data/processed/datos_limpios_track.csv", sep=";", encoding="latin-1")
df_his = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1"
)

In [4]:
df_track["hora_completa_cita"] = pd.to_datetime(df_track["hora_completa_cita"])
df_his["fecha_reserva"] = pd.to_datetime(df_his["fecha_reserva"])

In [5]:
print(
    f"La base de Track tiene citas desde {df_track['hora_completa_cita'].min()} "
    f"a {df_track['hora_completa_cita'].max()}"
)

print(
    f"La base de HIS tiene citas desde {df_his['fecha_reserva'].min()} a "
    f"{df_his['fecha_reserva'].max()}"
)
print()
print(f"La base de Track tiene {df_track.shape[0]} citas")
print(f"La base de HIS tiene {df_his.shape[0]} citas")

La base de Track tiene citas desde 2016-01-04 08:00:00 a 2022-12-30 20:30:00
La base de HIS tiene citas desde 2016-01-04 08:00:00 a 2022-12-30 14:50:00

La base de Track tiene 758350 citas
La base de HIS tiene 265441 citas


In [6]:
datos_pacientes_unicos_por_sesion = df_track.groupby(["id_paciente", "hora_completa_cita"]).head(1)

In [7]:
datos_pacientes_unicos_por_sesion

,sexo,edad,comuna,provincia,region,prevision,plan,tipoatencion,prestacion,fechacita,horacita,estadocita,hora_completa_cita,id_paciente,rango_etario
0,Hombre,71,Santiago,NaN,NaN,Grupo B,FONASA,Consulta Repetida,Consulta Integral De Especialidades En Medic I...,04/01/2016,14:30,Atendido,2016-01-04 14:30:00,f1b023d06c52e678b1b6707ac9e4d064cf7dc634e7e3aa...,"(70, 80]"
1,Hombre,90,Providencia,Santiago,Región Metropolitana,Grupo B,FONASA,Consulta Repetida,Consulta Integral De Especialidades En Medic I...,04/01/2016,14:30,Atendido,2016-01-04 14:30:00,4fa3f5adb335c1328cf02e29bc326bc411d04f5b354e93...,"(80, 90]"
2,Hombre,89,Macul,Santiago,Región Metropolitana,Grupo B,FONASA,Consulta Repetida,Consulta Integral De Especialidades En Medic I...,04/01/2016,14:45,Atendido,2016-01-04 14:45:00,43513afd69cf68073abba7acff2b0a26b341a8bd720a3e...,"(80, 90]"
3,Hombre,79,Rancagua,Cachapoal,Región del Libertador General Bernardo O?Higgins,Grupo B,FONASA,Consulta Repetida,Consulta Integral De Especialidades En Medic I...,04/01/2016,15:00,Atendido,2016-01-04 15:00:00,e77ef15b95ba1467b1d2e46eb4e0ade56000f71cf0aa45...,"(70, 80]"
4,Hombre,70,Providencia,Santiago,Región Metropolitana,Grupo C,FONASA,Consulta Repetida,Consulta Integral De Especialidades En Medic I...,04/01/2016,15:00,Atendido,2016-01-04 15:00:00,f94fdb45dff1aa80f284eea7e3789ae3cb9b252dfab70c...,"(60, 70]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758345,Hombre,63,Buin,NaN,NaN,Grupo B,FONASA,Consulta Repetida,Curacion Simple Ambulatoria,30/12/2022,13:00,Atendido,2022-12-30 13:00:00,d7ea6869a8ccc438d8af7c52efb67fe04b0b894f8e5251...,"(60, 70]"
758346,Hombre,73,La Florida,Santiago,Región Metropolitana,Grupo B,FONASA,Consulta Nueva,Curacion Compleja,30/12/2022,09:00,Atendido,2022-12-30 09:00:00,60dacc17adbaa7d35ffe17735a1794ed7f4737e7a09bfe...,"(70, 80]"
758347,Hombre,60,Las Condes,Santiago,Región Metropolitana,Grupo A,FONASA,Consulta Nueva,Curacion Compleja,30/12/2022,09:20,Atendido,2022-12-30 09:20:00,eb4b885901a0f7546044eafc36f0271cf6b1245b7e103a...,"(50, 60]"
758348,Hombre,63,Buin,NaN,NaN,Grupo B,FONASA,Consulta Nueva,Curacion Compleja,30/12/2022,12:00,Cancelado,2022-12-30 12:00:00,d7ea6869a8ccc438d8af7c52efb67fe04b0b894f8e5251...,"(60, 70]"


In [8]:
df_his_y_track = pd.merge(
    df_his,
    datos_pacientes_unicos_por_sesion,
    how="left",
    left_on=["id_paciente", "fecha_reserva"],
    right_on=["id_paciente", "hora_completa_cita"],
)

sin_datos_en_track = df_his_y_track[df_his_y_track.estadocita.isna()]

In [9]:
print(
    f"Luego de unir la base de HIS con Track, la primera ha quedado con "
    f"{df_his_y_track.shape[0]} consultas (de {df_his.shape[0]}). "
    f"Hay {sin_datos_en_track.shape[0]} consultas de HIS sin información en Track"
)

Luego de unir la base de HIS con Track, la primera ha quedado con 265441 consultas (de 265441). Hay 189 consultas de HIS sin información en Track


In [26]:
for paciente in sin_datos_en_track.id_paciente.unique():
    if paciente in df_track["id_paciente"]:
        print(f"{paciente} si esta")

Los resultados anteriores indican que ninguno de los pacientes faltantes se encuentra en la base
de Track (Y no ocurre debido a que los pacientes existen en la base, pero tienen una fecha de atencion distinta a la de HIS). 
Debido a lo anterior, estos pacientes carecen de datos sociodemográficos.